In [299]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk
import os
import numpy as np
import cv2
import mysql.connector

In [301]:
def generate_dataset():
    #if(e1.get()=="" or e2.get()==""):
        #messagebox.showinfo('Result', 'Please provide completed details of the user')
    #else:
    mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="amina2000",
        database="feel_me"
        )
    mycursor=mydb.cursor()
    mycursor.execute("SELECT * from authorized_user")
    myresult=mycursor.fetchall()
    id=1
    for x in myresult:
        id+=1
        
    sql="insert into authorized_user(id,username,password) values(%s,%s,%s)"
    val=(id,e1.get(),e2.get())
    mycursor.execute(sql,val)
    mydb.commit()
        
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5

        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result', 'Generating dataset completed!!')

<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\hp\AppData\Local\Temp/ipykernel_26572/766696361.py:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [302]:
def train_classifier():
    data_dir = "C:/Users/hp/Documents/Face recognizer/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training dataset completed')

In [303]:
def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )

            id, pred = clf.predict(gray_img[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            mydb=mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="amina2000",
                database="feel_me"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select username from authorized_user where id="+str(id))
            s = mycursor.fetchone()
            s = ''+''.join(s)
            global name
            if confidence>80:
                name = s
                cv2.putText(img, s, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                name = "UNKNOWN"
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

        return img, name

    # loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img, username = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
        cv2.imshow("face Detection", img)

        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()
    return username

In [304]:
def new_user():
    if(e1.get()=="" or e2.get()==""):
        messagebox.showinfo('Result', 'Please provide completed details of the user')
    else:
        generate_dataset()
        train_classifier()
        username = detect_face()
        w.destroy()
        home = Tk()
        home.title("Feel me APP")
        home.geometry("500x350")
        home.config(background="#A1C6E7")
        Label(home, text=username, font=("Courrier", 40), bg='#A1C6E7', fg='white').pack()
        home.mainloop()

In [305]:
def verify_password(given_username, entry):
    given_password=entry.get()
    if(given_password ==""):
        messagebox.showinfo('Result', 'Please provide a password')
    else:
        mydb=mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="amina2000",
                database="feel_me"
            )
        mycursor=mydb.cursor()
        sql = """SELECT password FROM `authorized_user` WHERE `username` = %s"""
        mycursor.execute(sql, (given_username,))
        password = mycursor.fetchone()
        password = ''+''.join(password)
        if(given_password != password):
            messagebox.showinfo('Result', 'Invalid password')
        else:
            login.destroy()
            home = Tk()
            home.title("Feel me APP")
            home.geometry("500x350")
            home.config(background="#A1C6E7")
            Label(home, text=given_username, font=("Courrier", 40), bg='#A1C6E7', fg='white').pack()
            home.mainloop()
        

In [306]:
def open_login(username):
    w.destroy()
    global login
    login = Tk()
    login.title("Feel me APP")
    login.geometry("350x500")
    login.resizable(0,0)
    login.config(background="#A1C6E7")

    Frame(login, width=250, height=400, bg = 'white').place(x=50, y=50)

    l3 = Label(login, text="Username", font=("consolas", 13), bg='white')
    l3.place(x=80, y=190)

    e3=Entry(login, width=20, border=0, font=1)
    e3.insert(END, username)
    e3.place(x=80, y=220)

    l4 = Label(login, text="Password", font=("consolas", 13), bg='white')
    l4.place(x=80, y=270)

    e4=Entry(login, width=20, border=0, font=1, show="*")
    e4.place(x=80, y=300)

    Frame(login, width=180, height=2, bg='#141414').place(x=80, y=240)
    Frame(login, width=180, height=2, bg='#141414').place(x=80, y=320)
    
    #global img1
    #img2 = ImageTk.PhotoImage((Image.open("user.png")).resize((100, 100)))
    #Label(image = img, border=0, justify=CENTER).place(x=120, y=60)

    #Button(w, text="Detect the face", width=20, height=2, bg='#A1C6E7', fg='white', command=generate_dataset).place(x=100, y=375)
    #Button(w, text="Detect the face", width=20, height=2, bg='#A1C6E7', fg='white', command=train_classifier).place(x=100, y=375)
    Button(login, text="Login", width=20, height=2, bg='#A1C6E7', fg='white', command=lambda given_username=username, entry=e4 : verify_password(given_username, entry)).place(x=100, y=345)
    

    login.mainloop()

In [307]:
def have_already_account():
    username = detect_face()
    if(username=="UNKNOWN"):
        messagebox.showinfo('Result', 'You are not an authorized user, please register first')
    else:
        open_login(username)
    #if
    #if(e2.get()==""):
        #messagebox.showinfo('Result', 'Please provide password')
    #else:
        

In [308]:
w = Tk()
w.title("Feel me APP")
w.geometry("350x500")
w.resizable(0,0)
w.config(background="#A1C6E7")

Frame(w, width=250, height=400, bg = 'white').place(x=50, y=50)

l1 = Label(w, text="Username", font=("consolas", 13), bg='white')
l1.place(x=80, y=190)

e1=Entry(w, width=20, border=0, font=1)
e1.place(x=80, y=220)

l2 = Label(w, text="Password", font=("consolas", 13), bg='white')
l2.place(x=80, y=270)

e2=Entry(w, width=20, border=0, font=1, show="*")
e2.place(x=80, y=300)

Frame(w, width=180, height=2, bg='#141414').place(x=80, y=240)
Frame(w, width=180, height=2, bg='#141414').place(x=80, y=320)

img = ImageTk.PhotoImage((Image.open("user.png")).resize((100, 100)))

Label(image = img, border=0, justify=CENTER).place(x=120, y=60)

#Button(w, text="Detect the face", width=20, height=2, bg='#A1C6E7', fg='white', command=generate_dataset).place(x=100, y=375)
#Button(w, text="Detect the face", width=20, height=2, bg='#A1C6E7', fg='white', command=train_classifier).place(x=100, y=375)
Button(w, text="New user", width=20, height=2, bg='#A1C6E7', fg='white', command=new_user).place(x=100, y=345)
Button(w, text="Have already an account", width=20, height=2, bg='#A1C6E7', fg='white', command=have_already_account).place(x=100, y=400)

w.mainloop()